In [ ]:
# Install necessary libraries
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils
!pip install pytesseract pillow sentence-transformers faiss-cpu transformers pdf2image pinecone-client

In [ ]:
import pytesseract  # OCR (Optical Character Recognition) tool to extract text from images
from PIL import Image  # Python Imaging Library to handle image files
from pdf2image import convert_from_path  # Convert PDF pages to images
from sentence_transformers import SentenceTransformer, util  # For sentence embeddings
from pinecone import Pinecone, ServerlessSpec  # Pinecone vector database for storing and querying embeddings
from tqdm.autonotebook import tqdm  # Progress bar for long loops
import numpy as np  # Numerical operations on arrays
from transformers import pipeline  # Huggingface's Transformers library for NLP tasks
import torch  # Deep learning library
import os  # Operating system interface
import pickle  # Serializing and deserializing Python objects

# Step 1: Extract text from PDF using OCR
def extract_text_from_pdf(pdf_path):
    try:
        # Convert PDF pages to images
        images = convert_from_path(pdf_path)
        text = ""
        # Extract text from each image using pytesseract
        for img in images:
            text += pytesseract.image_to_string(img)
        return text
    except Exception as e:
        # Handle any exceptions that occur
        print(f"An error occurred during PDF text extraction: {e}")
        return ""

# Step 2: Chunk the extracted text
def chunk_text(text, chunk_size=2000):
    # Split text into words
    words = text.split()
    chunks = []
    # Group words into chunks of specified size
    for i in range(0, len(words), chunk_size):
        chunks.append(' '.join(words[i:i + chunk_size]))
    return chunks

# Step 3: Generate embeddings for the chunks in batches
def generate_embeddings(chunks, model, batch_size=10):
    try:
        embeddings = []
        # Generate embeddings in batches to handle large data
        for i in tqdm(range(0, len(chunks), batch_size)):
            batch_chunks = chunks[i:i + batch_size]
            # Encode text chunks to embeddings
            batch_embeddings = model.encode(batch_chunks, convert_to_tensor=True)
            embeddings.append(batch_embeddings.cpu().numpy())
        # Combine all batch embeddings into a single array
        embeddings = np.vstack(embeddings)
        return embeddings
    except Exception as e:
        # Handle any exceptions that occur
        print(f"An error occurred during embedding generation: {e}")
        return None

# Step 4: Store embeddings in Pinecone
def store_embeddings_pinecone(index, embeddings, metadata):
    # Prepare vectors for upserting into Pinecone index
    vectors = [(str(i), embedding) for i, embedding in enumerate(embeddings)]
    # Upsert vectors into the Pinecone index
    index.upsert(vectors)

# Step 5: Create the chatbot function for knowledge-based queries
def chatbot_query(query, model, index, chunk_map):
    try:
        # Encode the query to get its embedding
        query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy().tolist()
        # Query Pinecone index to find similar embeddings
        results = index.query(vector=query_embedding, top_k=1)
        # Get the best matching response from the chunk map
        best_response = chunk_map[int(results['matches'][0]['id'])]
        return best_response
    except Exception as e:
        # Handle any exceptions that occur
        print(f"An error occurred during the chatbot query: {e}")
        return "Sorry, I couldn't find an answer."

# Step 6: Create the document QA function with sequence handling
def qa_response_with_context_split(query, context, qa_pipeline, max_length=768):
    try:
        # Get the QA model's response using the context
        response = qa_pipeline(question=query, context=context)
        return response['answer']
    except Exception as e:
        # Handle any exceptions that occur
        print(f"An error occurred during the document QA response: {e}")
        return "Sorry, I couldn't generate a response."

# Step 7: Use NER to enhance the chatbot’s understanding
def extract_entities(text, ner_pipeline):
    # Extract named entities from text using NER pipeline
    entities = ner_pipeline(text)
    return entities

# Step 8: Implement feedback mechanism
def get_feedback():
    while True:
        # Ask user for feedback
        feedback = input("Was the response helpful? (yes/no): ").strip().lower()
        if feedback in ['yes', 'no']:
            return feedback
        # If invalid feedback, prompt user again
        print("Please answer with 'yes' or 'no'.")

# Updated interactive_chatbot function
def interactive_chatbot(knowledge_model, index, chunk_map, qa_pipeline, ner_pipeline):
    print("Welcome to the improved interactive chatbot! Ask your questions below. Type 'exit' to end the session.")
    context = ""
    while True:
        # Get user query
        query = input("You: ")
        if query.lower() == 'exit':
            print("Goodbye!")
            break

        # Extract entities from the query
        entities = extract_entities(query, ner_pipeline)
        print(f"Extracted Entities: {entities}")

        # First try to get a response from the knowledge-based system
        knowledge_response = chatbot_query(query, knowledge_model, index, chunk_map)
        print(f"Knowledge-based Chatbot: {knowledge_response}")

        # Update context with the current query and response
        context += f"\nUser: {query}\nBot: {knowledge_response}"

        # Get a response from the QA model with context splitting
        qa_chatbot_response = qa_response_with_context_split(query, context, qa_pipeline)
        print(f"QA Chatbot: {qa_chatbot_response}")

        # Get feedback from the user
        feedback = get_feedback()
        print(f"Feedback: {feedback}")
        if feedback == "no":
            print("Sorry for the inconvenience. I'll try to improve.")

# Main function
def main(pdf_path, qa_model_name):
    # Extract text from PDF
    pdf_text = extract_text_from_pdf(pdf_path)
    print(f'There are {len(pdf_text)} characters in your document')

    # Chunk the extracted text
    chunks = chunk_text(pdf_text)

    # Print the chunks
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i}:\n{chunk}\n")

    # Initialize SentenceTransformer model for knowledge-based queries
    knowledge_model = SentenceTransformer('sentence-transformers/paraphrase-mpnet-base-v2')

    # Generate embeddings for the chunks in batches and save to disk
    chunk_embeddings = generate_embeddings(chunks, knowledge_model)

    if chunk_embeddings is None:
        print("Embedding generation failed. Exiting.")
        return

    # Save embeddings and chunks to disk
    with open('embeddings.pkl', 'wb') as f:
        pickle.dump(chunk_embeddings, f)

    with open('chunk_map.pkl', 'wb') as f:
        pickle.dump({i: chunk for i, chunk in enumerate(chunks)}, f)

    # Load embeddings and chunks from disk
    with open('embeddings.pkl', 'rb') as f:
        chunk_embeddings = pickle.load(f)

    with open('chunk_map.pkl', 'rb') as f:
        chunk_map = pickle.load(f)

    # Initialize Pinecone
    pc = Pinecone(api_key="8ea7eb99-cf4d-4ab3-a3a7-9d9057347011")
    if 'chatbot' not in pc.list_indexes().names():
        pc.create_index(
            name="chatbot",
            dimension=768,  # Replace with your model dimensions
            metric="cosine",  # Replace with your model metric
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
    index = pc.Index("chatbot")

    # Store embeddings in Pinecone
    store_embeddings_pinecone(index, chunk_embeddings, chunk_map)

    # Load pre-trained QA model pipeline
    qa_pipeline = pipeline("question-answering", model=qa_model_name)

    # Load pre-trained NER model pipeline
    ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

    # Start the interactive chatbot session
    interactive_chatbot(knowledge_model, index, chunk_map, qa_pipeline, ner_pipeline)

# Run the main function with the specified PDF path and model name
pdf_path = '/content/Pro Cv.pdf'
qa_model_name = 'deepset/roberta-large-squad2'  # Changed to a compatible QA model
main(pdf_path, qa_model_name)
